In [241]:
import numpy as np

In [242]:
def fit(x_train,y_train):
    result = {}
    result["total_data"]=len(y_train)
    key_val = set(y_train)
    for key in key_val:
        result[key] = {}
        currentclassrow = (y_train==key)
        x_train_current = x_train[currentclassrow]
        y_train_current = y_train[currentclassrow]
        numberoffeatures = x_train.shape[1]
        for feature in range(1,1+numberoffeatures):
            result[key][feature]={}
            val = set(x_train[:,feature-1])
            for indval in val:
                result[key][feature][indval] = (x_train_current[:,feature-1]==indval).sum()
        result[key]["count"]=len(y_train_current)
    return result

In [256]:
def prob(dictionary,x,curr_class):
    output = np.log(1)
    #it has 2 components (prioir prob / class prob)
    classprob = np.log(dictionary[curr_class]["count"])-np.log(dictionary["total_data"])
    #now we do 2nd component
    numoffeatures = len(dictionary[curr_class].keys())-1
    for i in range(1,1+numoffeatures):
        xi = x[i-1]
        countwithxi = dictionary[curr_class][i][xi] +1
        prob = np.log(countwithxi)-np.log(dictionary[curr_class]["count"] +len(dictionary[curr_class][i].keys()))
        output +=  prob
    #print (output,classprob)
    return output+classprob

In [257]:
def predictsinglepoint(dictionary,x):
    classes = dictionary.keys()
    bestprob = -1000
    bestclass = -1
    for curr_class in classes:
        if(curr_class== "total_data"):
            continue
        p_curr_class = prob(dictionary, x, curr_class)
        if(p_curr_class>bestprob):
            bestprob = p_curr_class
            bestclass= curr_class
    return bestclass

In [258]:
def predict(dictionary , x_test):
    y_pred = []
    for x in x_test:
        x_class = predictsinglepoint(dictionary,x)
        y_pred.append(x_class)
        #print(x_class)
    return y_pred

In [259]:
def makelabel(column):
    seclim = column.mean()
    firlim = seclim / 2
    thirlim = firlim * 3 
    for i in range(len(column)):
        if(column[i]<firlim):
            column[i] = 0
        elif(column[i]>=firlim and column[i]<seclim):
            column[i]=1
        else:
            column[i]=2
    return column

In [260]:
from sklearn import datasets
db = datasets.load_iris()
x = db.data
y = db.target

In [261]:
for i in range(x.shape[-1]):
    x[:,i] = makelabel(x[:,i])

In [274]:
from sklearn import model_selection
x_train , x_test , y_train , y_test = model_selection.train_test_split(x,y)

In [275]:
print(x_train.shape)

(112, 4)


In [276]:
dictionary = fit(x_train,y_train)

In [280]:
print(dictionary[1])

{1: {1.0: 18, 2.0: 17}, 2: {1.0: 28, 2.0: 7}, 3: {0.0: 0, 1.0: 5, 2.0: 30}, 4: {0.0: 0, 1.0: 8, 2.0: 27}, 'count': 35}


In [278]:
y_pred = predict(dictionary,x_test)
print(y_pred)

[2, 2, 2, 0, 2, 2, 2, 2, 0, 0, 0, 0, 2, 2, 1, 0, 0, 1, 2, 0, 1, 0, 2, 1, 0, 2, 0, 2, 0, 2, 0, 2, 2, 0, 1, 1, 2, 2]


In [279]:
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        14
           1       1.00      0.40      0.57        15
           2       0.50      1.00      0.67         9

   micro avg       0.76      0.76      0.76        38
   macro avg       0.83      0.80      0.75        38
weighted avg       0.88      0.76      0.75        38

[[14  0  0]
 [ 0  6  9]
 [ 0  0  9]]
